In [1]:
import torch
from frames.representations import FrameUnembeddingRepresentation

In [2]:
fur = FrameUnembeddingRepresentation.from_model_id(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-02-26 13:48:59.663 | INFO     | frames.models.hf.base:__init__:88 - Loaded model: meta-llama/Meta-Llama-3.1-8B-Instruct
2025-02-26 13:48:59.665 | WARNING  | frames.models.hf.base:__init__:89 - memory cost: 15316 Mb


In [3]:
df = fur.data.get_dataframe(fur.model.tokenizer, min_lemmas_per_synset=11, max_token_count=3)

df2 = df.explode("tokens")["tokens"].value_counts().reset_index().to_pandas()

df2["symbol"] = df2["tokens"].apply(lambda id: fur.model.tokenizer.decode([id]))

df2[df2["symbol"].str.contains("ism")]

# df2

,tokens,count,symbol
130,17434,1137,ismo
218,2191,828,ism
235,44618,766,isme
1066,117324,222,ismus
4062,39252,67,isma
...,...,...,...
37667,90674,2,conservatism
38044,55280,2,activism
38193,90442,2,patriotism
38345,84581,2,liberalism


In [4]:
df[df["tokens"].list.index(2191) != -1]["synset"].value_counts()

synset
dictatorship.n.01    17
seismology.n.01      13
seismic.s.01         12
absolutism.n.03      10
ableism.n.01         10
                     ..
appalling.s.01        1
mechanism.n.02        1
depress.v.01          1
dismember.v.02        1
conservatism.n.01     1
Name: count, Length: 245, dtype: int64

In [24]:
df[df["tokens"].list.index(2191) != -1]

,synset,lemma,pos,lang,is top noun,tokens,token count,synset id,lemma id
27757,ableism.n.01,ableism,n,eng,False,"[481, 2191, 220]",3,84,231047
27757,ableism.n.01,ableism,n,eng,False,"[3025, 2191, 128009]",2,84,33045
27757,ableism.n.01,ableism,n,eng,False,"[3025, 2191, 220]",3,84,33046
27757,ableism.n.01,ableism,n,eng,False,"[481, 2191, 128009]",2,84,231046
27757,ableism.n.01,ablism,n,eng,False,"[62573, 2191, 220]",3,84,231053
...,...,...,...,...,...,...,...,...,...
67958,zionism.n.02,Zionism,n,eng,False,"[41435, 2191, 128009]",2,34612,32457
67958,zionism.n.02,Zionism,n,eng,False,"[41435, 2191, 220]",3,34612,32458
67958,zionism.n.02,Zionism,n,eng,False,"[57, 290, 2191]",3,34612,230481
25615,zoophilia.n.01,zoophilism,n,eng,False,"[19263, 98635, 2191]",3,34632,192544


In [ ]:
# get concept frames that rely on "ism" token for max_token_count= 1 and 3
# compare the UMAP embeddings of each set, check if they are separable
# to input frame into UMAP, flatten as a vector and pad with zeros or compute MDS (slow)